In [4]:
import random

import numpy as np

# Tkinter库， Python 中用于创建图形用户界面(GUI)
# tk：提供创建窗口、按钮、标签等图形界面组件的基本功能 
import tkinter as tk
# messagebox：显示各种标准对话框
from tkinter import messagebox
# askcolor: 弹出颜色选择对话框，让用户选择颜色
from tkinter.colorchooser import askcolor

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.ticker import MaxNLocator

In [5]:
class MultiAreaCircleApp:
    def __init__(self, master):
        self.master     = master
        self.max_active = 3 # 限制最大激活数量
        self.grid_size = 100

        # 数据层
        # Create 8 margins, each with 2 random circular areas
        # 随机生成 margin 的形状
        self.margins = []
        min_r, max_r = 1, self.grid_size // 4
        for _ in range(8):
            areas = []
            for __ in range(2):
                cx = random.randint(0, self.grid_size-1)
                cy = random.randint(0, self.grid_size-1)
                r  = random.randint(min_r, max_r)
                areas.append((cx, cy, r))
            self.margins.append(areas)


        # UI Layer
        # Build a 100×100 grid of coordinates
        # 构建 100×100 的网格坐标
        
        xs = np.arange(self.grid_size)
        ys = np.arange(self.grid_size)
        self.X, self.Y = np.meshgrid(xs, ys)

        # Slot and overlap colors
        self.slot_colors = [(0,0,255),(255,0,0),(0,255,0)]
        self.overlap_colors = {(0,1):(128,0,128),(0,2):(165,42,42),(1,2):(255,255,0)}
        self.overlap_color_names = {(0,1):'purple',(0,2):'brown',(1,2):'yellow'}
        self.triple_color = (0,0,0)

        # UI state
        self.vars = []
        self.checkbuttons = []
        self.slot_assignment = {}

        # Use PanedWindow to lock left panel size
        pane = tk.PanedWindow(master, orient='horizontal')
        pane.pack(fill='both', expand=True)

        # Left control pane (fixed width)
        ctrl = tk.Frame(pane, width=250)
        pane.add(ctrl, sticky='ns')
        ctrl.pack_propagate(False)
        ctrl.grid_propagate(False)

        # Checkboxes
        for i, areas in enumerate(self.margins):
            var = tk.BooleanVar(master=ctrl, value=False)
            cb = tk.Checkbutton(ctrl,
                                text=f"Margin {i+1} ({len(areas)} areas)",
                                variable=var,
                                font=('Arial',12),
                                command=lambda i=i: self.on_toggle(i))
            cb.grid(row=i, column=0, sticky='w', pady=2)
            self.vars.append(var)
            self.checkbuttons.append(cb)

        # Overlap message label
        self.msg = tk.Label(ctrl, text='', justify='left', font=('Arial',12))
        self.msg.grid(row=len(self.margins), column=0, sticky='w', pady=(10,0))

        # Right canvas pane (resizable)
        canvas_container = tk.Frame(pane)
        pane.add(canvas_container, sticky='nsew')

        # Matplotlib figure and canvas
        self.fig = plt.Figure(figsize=(8,8))
        self.ax  = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=canvas_container)
        self.canvas.get_tk_widget().pack(fill='both', expand=True)

        # Navigation toolbar
        self.toolbar = NavigationToolbar2Tk(self.canvas, canvas_container)
        self.toolbar.update()
        self.toolbar.pack(fill='x')

        # Status bar for coords
        self.status = tk.Label(canvas_container, text='Coords: ( , )', anchor='w', font=('Arial',10))
        self.status.pack(side='bottom', fill='x')
        self.canvas.mpl_connect('motion_notify_event', self.on_motion)

        # Initial draw
        self.on_toggle(None)

    def on_toggle(self, idx):
        if idx is not None:
            if self.vars[idx].get():
                if len(self.slot_assignment) < self.max_active:
                    for slot in range(self.max_active):
                        if slot not in self.slot_assignment.values():
                            self.slot_assignment[idx] = slot
                            break
                else:
                    self.vars[idx].set(False)
            else:
                self.slot_assignment.pop(idx, None)
        active = len(self.slot_assignment)
        for j, cb in enumerate(self.checkbuttons):
            if self.vars[j].get(): cb.config(state=tk.NORMAL)
            else: cb.config(state=tk.NORMAL if active<self.max_active else tk.DISABLED)
        self.update_message()
        self.draw()

    def update_message(self):
        rev = {slot:midx for midx,slot in self.slot_assignment.items()}
        slots = sorted(rev.keys())
        lines=[]
        for i in range(len(slots)):
            for j in range(i+1,len(slots)):
                a,b=slots[i],slots[j]
                lines.append(f"Overlap M{rev[a]+1}&M{rev[b]+1}: {self.overlap_color_names.get((a,b),'')}")
        if len(slots)==3: lines.append('Triple-overlap: black')
        self.msg.config(text='\n'.join(lines))

    def draw(self):
        # 1. 准备画布与槽位掩码
        H,W=self.grid_size,self.grid_size
        img=np.ones((H,W,4),dtype=np.uint8)*255
        masks={s:np.zeros((H,W),bool) for s in range(self.max_active)}
        
        # 2. 把“已激活的 margin”并入对应槽位的掩码
        for midx,slot in self.slot_assignment.items():
            for cx,cy,r in self.margins[midx]:
                masks[slot]|=((self.X-cx)**2+(self.Y-cy)**2)<=r*r
        m0,m1,m2=masks[0],masks[1],masks[2]
        single0=m0&~m1&~m2; single1=m1&~m0&~m2; single2=m2&~m0&~m1
        pair01=m0&m1&~m2; pair02=m0&m2&~m1; pair12=m1&m2&~m0; triple=m0&m1&m2
        img[single0,:3]=self.slot_colors[0]; img[single0,3]=255
        img[single1,:3]=self.slot_colors[1]; img[single1,3]=255
        img[single2,:3]=self.slot_colors[2]; img[single2,3]=255
        img[pair01,:3]=self.overlap_colors[(0,1)]; img[pair01,3]=255
        img[pair02,:3]=self.overlap_colors[(0,2)]; img[pair02,3]=255
        img[pair12,:3]=self.overlap_colors[(1,2)]; img[pair12,3]=255
        img[triple,:3]=self.triple_color; img[triple,3]=255

        # 5. 绘图、坐标轴与网格
        self.fig.subplots_adjust(right=0.75)
        self.ax.clear(); self.ax.imshow(img,interpolation='nearest')
        self.ax.xaxis.set_major_locator(MaxNLocator(nbins=10,integer=True))
        self.ax.yaxis.set_major_locator(MaxNLocator(nbins=10,integer=True))
        self.ax.grid(True,color='gray',linestyle='-',linewidth=0.5)
        self.ax.set_title(f"{len(self.slot_assignment)} active margins")

        from matplotlib.patches import Patch
        rev={slot:midx for midx,slot in self.slot_assignment.items()}
        handles=[]
        # margins
        for slot,midx in sorted(rev.items()):
            col=tuple(c/255 for c in self.slot_colors[slot])
            handles.append(Patch(facecolor=col,edgecolor='black',label=f"Margin {midx+1}"))
        # overlaps
        if pair01.any(): handles.append(Patch(facecolor=tuple(c/255 for c in self.overlap_colors[(0,1)]),edgecolor='black',label=f"Overlap M{rev[0]+1}&M{rev[1]+1}"))
        if pair02.any(): handles.append(Patch(facecolor=tuple(c/255 for c in self.overlap_colors[(0,2)]),edgecolor='black',label=f"Overlap M{rev[0]+1}&M{rev[2]+1}"))
        if pair12.any(): handles.append(Patch(facecolor=tuple(c/255 for c in self.overlap_colors[(1,2)]),edgecolor='black',label=f"Overlap M{rev[1]+1}&M{rev[2]+1}"))
        if triple.any(): handles.append(Patch(facecolor=tuple(c/255 for c in self.triple_color),edgecolor='black',label="Triple overlap"))
        if handles: self.ax.legend(handles=handles,title='Legends',loc='upper left',bbox_to_anchor=(1.02,1),borderaxespad=0)
        self.canvas.draw()

    def on_motion(self,event):
        if event.inaxes==self.ax and event.xdata is not None and event.ydata is not None:
            self.status.config(text=f"Coords: ({event.xdata:.2f}, {event.ydata:.2f})")
        else:
            self.status.config(text='Coords: ( , )')


In [6]:
if __name__=='__main__':
    root=tk.Tk(); root.title('Interactive Margins (max 3 selectable)')
    app=MultiAreaCircleApp(root)
    root.mainloop()